In [1]:
import findspark
findspark.init()
import pandas as pd;
import csv;
import numpy as np;
from pyspark import SparkContext
from pyspark import SparkConf
import re

In [2]:
conf = SparkConf().setAppName('Read & parse text data file in pyspark')
sc = SparkContext.getOrCreate(conf=conf)

In [3]:
lines = sc.textFile('./assets/AMiner-Author.txt').zipWithIndex()

In [4]:
# Find the starting row of each data entry
pos = lines.filter(lambda x: "#index" in x[0]).map(lambda x: x[1]).collect()

In [5]:
zeroArray = np.zeros(lines.count(), dtype=int)

In [6]:
def exchangeElementAtIndexToOne(index):
    zeroArray[index] = 1;

In [7]:
for element in pos:
    exchangeElementAtIndexToOne(element)

In [8]:
# Calculate cumulative sum of starting data entry indexes
summedArray = np.cumsum(zeroArray)

In [9]:
broadcastedArray = sc.broadcast(summedArray);

In [10]:
def createBroarcastedTuple(x):
    return (broadcastedArray.value[x[1]], x[0])

In [11]:
papers = lines.map(lambda x: createBroarcastedTuple(x))

In [12]:
list_get = lambda l, x, d=None: d if not l[x:x+1] else l[x] # safe getter of list values

paper_keys = ["#index", "#*", "#@", "#o", "#t", "#c", "#%", "#!"]
author_keys = ["#index", "#n", "#a", "#pc", "#cn", "#hi", "#pi", "#upi", "#t"]

AUTHOR_KEYS_MAP = {
    "#index": "id", "#n": "name", "#a": "affiliations", "#pc": "paper_count", "#cn": "citation_count", "#t": "research_interests", "#hi": "h_index", "#upi": "u_p_index", "#pi": "p_index"
}

with open('./assets/parsedData/authors.csv', 'w', encoding='utf8', newline='') as output_file:
    fc = csv.DictWriter(output_file, fieldnames=author_keys)
    fc.writeheader()
#     fc.writerows(mappedPaperDicts.collect())

def splitIntoKeyValue(stringToSplit):
    if type(stringToSplit) is str:
        if len(stringToSplit) > 0:
            stringToSplit = stringToSplit.split(" ", 1)
            mappedKey = AUTHOR_KEYS_MAP[list_get(stringToSplit, 0)]
#             print(">>> ", mappedKey)
            if (mappedKey):
                stringToSplit = { mappedKey: list_get(stringToSplit, 1, '') }
            else:
                stringToSplit = {}
        else:
            stringToSplit = {}
    return stringToSplit

def reduceFeaturesToDict(featureA={}, featureB=''):
#     print("here")
    try:
        splittedA = splitIntoKeyValue(featureA);
        splittedB = splitIntoKeyValue(featureB);
#         print("->>>>-->",splittedA)
#         if type(splittedA) is str:
#             if len(splittedA) > 0:
#                 splittedA = splittedA.split(" ", 1)
#                 mappedKey = AUTHOR_KEYS_MAP[list_get(splittedA, 0)]
#                 if (mappedKey):
#                     splittedA = { mappedKey: list_get(splittedA, 1, '') }
#                 else:
#                     splittedA = {}
#             else:
#                 splittedA = {}

#         splittedB = featureB;
#         if type(featureB) is str:
#             if len(featureB) > 0:
#                 splittedB = featureB.split(" ", 1)
#                 splittedB = { list_get(splittedB, 0): list_get(splittedB, 1, '') }
#             else:
#                 splittedB = {}
#         if 
        return { **splittedA, **splittedB }
    except:
        print("ERROR: ", featureA, featureB)
        raise

In [13]:
reducedPapers = papers.reduceByKey(lambda a, b: reduceFeaturesToDict(a, b))

In [14]:
print(reducedPapers.sortByKey(ascending=False).first())

herehere:>                                                         (0 + 8) / 17]
>>>  name
>>>  affiliations
->>>>--> 
{'name': ' Vinayak'}
>>>  hereu_p_index

>>>  paper_count
->>>>-->>>>   research_interests
{'name': ' Vinayak', 'affiliations': 'Purdue University, West Lafayette, Indiana, USA'}->>>>-->
 here
>>>  citation_count
->>>>--> here{'name': ' Vinayak', 'affiliations': 'Purdue University, West Lafayette, Indiana, USA', 'paper_count': '3'}
here
{'u_p_index': '0.0000'}
>>>  h_index
->>>>--> {'name': ' Vinayak', 'affiliations': 'Purdue University, West Lafayette, Indiana, USA', 'paper_count': '3', 'citation_count': '5'}
here
->>>>--> {'u_p_index': '0.0000', 'research_interests': 'Kazakh vowel;Acoustic Database;Intelligent Voice System;Formant Analysis;Mathematical Model;Kazakh VowelsThe traditional classification;Kazakh phonetics;Kazakh telephone speech;new natural classification system;acoustic model'}
here
>>>  p_index
->>>>-->here 
{'name': ' Vinayak', 'affiliations': 'Purdue

{'id': '212245', 'name': 'Tang Jian', 'affiliations': '', 'paper_count': '1', 'citation_count': '0', 'h_index': '0', 'p_index': '0.0000'}
here
 here
here
->>>>-->id>>> 
 research_interests

->>>>--> >>> >>>   u_p_index 
name->>>>--> {'id': '314986', 'name': 'H. Ichikawa', 'affiliations': '', 'paper_count': '1', 'citation_count': '0', 'h_index': '0', 'p_index': '0.0000', 'u_p_index': '0.0000', 'research_interests': 'DAT drive system;R-DATDigital audio tape;compact disk player;drive mechanism;recording head;recording media;Positioning system;high-precision positioning;light weight mechanism;mechanical problem'}
{'id': '421170', 'name': 'Hiroyuki Daizo', 'affiliations': 'Shizuoka University, Shizuoka, Japan', 'paper_count': '1', 'citation_count': '0', 'h_index': '0', 'p_index': '0.0000'}

here{'id': '212245', 'name': 'Tang Jian', 'affiliations': '', 'paper_count': '1', 'citation_count': '0', 'h_index': '0', 'p_index': '0.0000', 'u_p_index': '0.0000'}

here
>>>  id
>>>  name
->>>>--> {'id'

KeyboardInterrupt: 

In [14]:
mappedPaperDicts = reducedPapers.map(lambda x: x[1]) # retrieve dicts from the tuples

In [15]:
df = sqlContext.createDataFrame(mappedPaperDicts)

/usr/local/Cellar/apache-spark/3.2.0/libexec/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/usr/local/Cellar/apache-spark/3.2.0/libexec/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/usr/local/Cellar/apache-spark/3.2.0/libexec/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/usr/local/Cellar/apache-spark/3.2.0/libexec/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/usr/local/Cellar/apache-spark/3.2.0/libexec/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/usr/local/Cellar/apache-spark/3.2.0/libexec/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/usr/local/Cellar/apac

22/01/03 22:56:24 WARN TaskSetManager: Lost task 4.0 in stage 3.0 (TID 55) (192.168.0.171 executor driver): TaskKilled (Stage cancelled)
22/01/03 22:56:24 WARN TaskSetManager: Lost task 3.0 in stage 3.0 (TID 54) (192.168.0.171 executor driver): TaskKilled (Stage cancelled)


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 3.0 failed 1 times, most recent failure: Lost task 7.0 in stage 3.0 (TID 58) (192.168.0.171 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/3.2.0/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/usr/local/Cellar/apache-spark/3.2.0/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 609, in process
    out_iter = func(split_index, iterator)
  File "/usr/local/Cellar/apache-spark/3.2.0/libexec/python/pyspark/rdd.py", line 2918, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/Cellar/apache-spark/3.2.0/libexec/python/pyspark/rdd.py", line 2918, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/Cellar/apac

In [ ]:
# Save data to csv file
df.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("mydata.csv")

22/01/03 22:56:24 WARN TaskSetManager: Lost task 5.0 in stage 3.0 (TID 56) (192.168.0.171 executor driver): TaskKilled (Stage cancelled)
22/01/03 22:56:24 WARN TaskSetManager: Lost task 2.0 in stage 3.0 (TID 53) (192.168.0.171 executor driver): TaskKilled (Stage cancelled)
22/01/03 22:56:24 WARN TaskSetManager: Lost task 1.0 in stage 3.0 (TID 52) (192.168.0.171 executor driver): TaskKilled (Stage cancelled)
22/01/03 22:56:24 WARN TaskSetManager: Lost task 6.0 in stage 3.0 (TID 57) (192.168.0.171 executor driver): TaskKilled (Stage cancelled)
22/01/03 22:56:24 WARN TaskSetManager: Lost task 0.0 in stage 3.0 (TID 51) (192.168.0.171 executor driver): TaskKilled (Stage cancelled)


In [ ]:
mappedPaperDicts.saveAsTextFile('./assets/parsedData/authors')